In [12]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [13]:
token = 'vk1.a.MZGEKhed1e58okSLuG_Yx8kUTMQm7jKkKn3WQzG_sa1y0a-voHbjGYV-RUfJl5TUcXJkpCsiixv1Bz1Y2U4EDwuC3kvKEyF4swZI1GqVUEpDOKrhRW40h_Ur6l1jJCXJMAe5EumxDBDeGLEp4eVJ1x5qBc9yciIrPPmTD54k6Kx5EaV8P8d2ZLE0QvLbE2awyeKkPVilbJRsEw-UhTEaOg'

In [14]:
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

In [18]:
vk.messages.send(
    chat_id=1,
    random_id=4425,
    message='Так, работаем')

7

In [19]:
df = pd.read_csv('ads_data.csv.zip', compression='zip')

In [20]:
ad_data = df.groupby(['ad_id', 'event'], as_index=False) \
    .agg({'user_id': 'count'})

In [21]:
ad_data = ad_data.pivot(index='ad_id', columns='event', values='user_id').reset_index()

In [22]:
ad_data = ad_data.fillna(0).assign(ctr=ad_data.click / ad_data.view)

In [23]:
top_ctr = ad_data.query('click > 20 & view > 100').sort_values('ctr').tail(10)

In [24]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

In [25]:
path = 'top_ctr_data.csv'
file_name = 'top_ctr_data.csv'

path_to_file = path
upload_url = vk.docs.getMessagesUploadServer(peer_id=2000000001)["upload_url"]
file = {'file': (file_name, open(path_to_file, 'rb'))}

response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)
json_data

{'file': '-217155500|0|-1|237331|0431c0d652|csv|403|top_ctr_data.csv|c2f9e346fc49058f43500d24dbf7ae44|844d1bcdb39c66994b06aadfa8e15951||||eyJkaXNrIjozNywicGVlcl9zZW5kZXIiOiItMjE3MTU1NTAwIn0='}

In [26]:
saved_file = vk.docs.save(file=json_data['file'], title=file_name)

In [27]:
saved_file

{'type': 'doc',
 'doc': {'id': 646947768,
  'owner_id': -217155500,
  'title': 'top_ctr_data.csv',
  'size': 403,
  'ext': 'csv',
  'date': 1668348123,
  'type': 1,
  'url': 'https://vk.com/doc-217155500_646947768?hash=HzDcbQfW6le1jaMBYXhzOhz0qtRNOUYLHKcx5gpzqTX&dl=FUZDCNZRGU2TKMBQ:1668348123:sKWnZjF34vTLEjmAtZFz1Qdpv0p7tA9xzc2HiJfN354&api=1&no_preview=1'}}

In [28]:
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])

In [29]:
attachment

'doc-217155500_646947768'

In [30]:
vk.messages.send(
    chat_id=1,
    random_id=3,
    message='Топ объявлений по CTR', 
    attachment = attachment
)

8

In [35]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [36]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

my_mail = 'anatoly1804@gmail.com'

# Authorization
credentials = ServiceAccountCredentials.from_json_keyfile_name('heroic-venture-268009-1dfbcc34e5fa.json', scope)
gs = gspread.authorize(credentials)

FileNotFoundError: [Errno 2] No such file or directory: 'heroic-venture-268009-1dfbcc34e5fa.json'

In [60]:
# Name of the table in google sheets, 
# can be url for open_by_url
# or id (key) part for open_by_key
table_name = 'to_sequence'  # Your table
# Get this table
work_sheet = gs.open(table_name)

In [72]:
# Select 1st sheet
sheet1 = work_sheet.sheet1

# Get data in python lists format
data = sheet1.get_all_values()

In [73]:
data

[['price', 'group'],
 ['110', 'Group_1'],
 ['20', 'Group_2'],
 ['250', 'group_3'],
 ['500', 'group_4']]

In [74]:
headers = data.pop(0)

In [75]:
df = pd.DataFrame(data, columns=headers)

In [76]:
df

,price,group
0,110,Group_1
1,20,Group_2
2,250,group_3
3,500,group_4


In [67]:
df.sort_values('price', ascending=False)

,price,group
1,20,Group_2
0,110,Group_1


In [71]:
sheet1.append_row([500, 'group_4'])

{'spreadsheetId': '1gjyC_l8LMR3xq6h9w0GlbfsCpjl_V1j3dl9_0WxYyq8',
 'tableRange': "'Лист1'!A1:B4",
 'updates': {'spreadsheetId': '1gjyC_l8LMR3xq6h9w0GlbfsCpjl_V1j3dl9_0WxYyq8',
  'updatedRange': "'Лист1'!A5:B5",
  'updatedRows': 1,
  'updatedColumns': 2,
  'updatedCells': 2}}

In [77]:
# Looks like spreadsheet should be already present at the dist (so, run code in create table section)
spreadsheet_name = 'A new spreadsheet'
sheet = 'KarpovCorses2'
d2g.upload(df, spreadsheet_name, sheet, credentials=credentials, row_names=True)

<Worksheet 'KarpovCorses2' id:1323350636>

In [84]:
url = 'https://api-metrika.yandex.net/stat/v1/data?'
visits = 'metrics=ym:s:visits&dimensions=ym:s:date&id=44147844'

In [85]:
vistis_url = url + visits

In [86]:
vistis_request = requests.get(vistis_url)

In [87]:
vistis_request

<Response [200]>

In [88]:
json_data = json.loads(vistis_request.text)

In [90]:
json_data['data']

[{'dimensions': [{'name': '2020-06-05'}], 'metrics': [718.0]},
 {'dimensions': [{'name': '2020-06-02'}], 'metrics': [644.0]},
 {'dimensions': [{'name': '2020-06-03'}], 'metrics': [633.0]},
 {'dimensions': [{'name': '2020-06-04'}], 'metrics': [606.0]},
 {'dimensions': [{'name': '2020-06-01'}], 'metrics': [572.0]},
 {'dimensions': [{'name': '2020-06-06'}], 'metrics': [375.0]},
 {'dimensions': [{'name': '2020-06-07'}], 'metrics': [176.0]}]

In [95]:
y_df = pd.DataFrame([(i['dimensions'][0]['name'], 
  i['metrics'][0]) for i in json_data['data']], columns=['date', 
                                                        'visits'])

In [96]:
spreadsheet_name = 'A new spreadsheet'
sheet = 'Yandex_visits'
d2g.upload(y_df, spreadsheet_name, sheet, credentials=credentials, row_names=True)

<Worksheet 'Yandex_visits' id:197715693>